In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [1]:
import requests

In [2]:
import ipyparallel as ipp

In [40]:
def counter(n):
    total = 0
    for i in range(1,n+1):
        total += i
    return(total)

In [43]:
def echo(x):
    return(x)

In [38]:
def sumv2(x):
    return(sum(x))

In [31]:
import numpy as np

In [44]:
rc = ipp.Client()
print(rc.ids)

[0, 1, 2, 3]


In [45]:
lv = rc.load_balanced_view()

In [48]:
lv.map_sync(counter, range(10))

[0, 1, 3, 6, 10, 15, 21, 28, 36, 45]

In [25]:
dv.wait()

True

In [10]:
results[0]

CompositeError: one or more exceptions from call to method: <lambda>
[2:apply]: NameError: name 'counter' is not defined
[0:apply]: NameError: name 'counter' is not defined
[1:apply]: NameError: name 'counter' is not defined
[3:apply]: NameError: name 'counter' is not defined

## Session Information

In [7]:
# Session Values
with open("data/ncga_sessions_html", 'r') as f:
    session_html = "".join(f.readlines())

In [10]:
print(session_html[:100])

<option value="2015E4">2016 Extra Session 4</option>
<option value="2015E3">2016 Extra Session 3</op


In [11]:
session_soup = bs(session_html, 'html.parser')

In [18]:
sessions = session_soup.find_all('option')
sessions = [{"query_value" : session['value'], "name" : session.text.strip()} \
            for session in sessions]
print(len(sessions))
sessions[0]

33


{'name': '2016 Extra Session 4', 'query_value': '2015E4'}

## Query Stuff

In [53]:
query_root = "http://www.ncleg.net/gascripts/"

**Example 01:**

> http://www.ncleg.net/gascripts/BillSearch/BillSearch.asp?typeToSearch=ALL&chamberToSearch=&sessionToSearch=2011&searchCriteria=filed&sortBy=relevance

In [22]:
query_attrs = ['typeToSearch=',
               'chamberToSearch=',
               'sessionToSearch=',
               'searchCriteria=',
               'sortBy=']

In [44]:
url = query_root + "type=json&" + \
"&".join([query_attrs[0] + "ALL" ,
          query_attrs[1] + "",
          query_attrs[2] + "2015",
          query_attrs[3] + "filed",
          query_attrs[4] + "relevance"])

In [45]:
url

'http://www.ncleg.net/gascripts/BillSearch/BillSearch.asp?type=json&typeToSearch=ALL&chamberToSearch=&sessionToSearch=2015&searchCriteria=filed&sortBy=relevance'

**Example 02:**

The first is the info page about the bill, and the second is Version 1 of the bill in HTML format

> http://www.ncleg.net/gascripts/BillLookUp/BillLookUp.pl?Session=2015&BillID=H123
> http://www.ncleg.net/Sessions/2015/Bills/House/HTML/H123v1.html

Method 2 is "cleaner" and a bit easier.  Don't need to worry about constructing a query that returns all of the Bills filed during a session.  Simply interate over URLs until we fail to retrieve pages (multiple times.  Stopping criteria can be interesting here...)

In [77]:
def buildMethod02URL(session, bill_id):
    url = query_root + "BillLookUp/BillLookUp.pl?" +\
          "Session=" + str(session) + "&" +\
          "BillID=" + str(bill_id)
    return(url)

In [78]:
url = buildMethod02URL(2015, 'H1')
url

'http://www.ncleg.net/gascripts/BillLookUp/BillLookUp.pl?Session=2015&BillID=H1'

In [79]:
bill_page = requests.get(url)

In [80]:
bill_page.status_code

200

In [81]:
bill_soup = bs(bill_page.content, "html.parser")

In [93]:
tables = bill_soup.find_all("table")
print(len(tables))

18


In [94]:
for t in tables:
    if t.text.strip().startswith("View Available Bill Summaries"):
        bill_content_table = t
        print(t)

<table align="center" cellpadding="0" cellspacing="0" style="margin-right:0px; margin-top: 10px; margin-bottom: 10px; width: 100%;">
<tr>
<td style="vertical-align: top; width: 35%; border: 1px solid #999;padding:0px;">
<table cellspacing="0" style="width: 100%; margin: 0px; padding: 0px;">
<tr><td colspan="2" style="background-color: #ced0d2; padding: 0px; text-align: center; font-weight: bold;border-bottom: 1px solid #555;border-top: 1px solid #555;"><a href="/gascripts/billsummaries/billsummaries.pl?Session=2015&amp;BillID=H1">View Available Bill Summaries</a> </td></tr>
<tr class="sectionBG1">
<th class="tableHeader" style="border-bottom: 1px solid #555; padding-left: 5px;">Bill Text</th><th class="tableHeader" nowrap="" style="border-bottom: 1px solid #555;">Fiscal Note</th>
</tr>
<tr>
<td nowrap="" style="border-bottom: 1px dotted #DCDCDC; padding-left:4px; padding-bottom:4px; vertical-align: top;"><a href="/Sessions/2015/Bills/House/PDF/H1v0.pdf">Filed</a> <span style="font: 9px

In [105]:
for row in bill_content_table.find_all("tr"):
    print(row.find_all('a'))

[<a href="/gascripts/billsummaries/billsummaries.pl?Session=2015&amp;BillID=H1">View Available Bill Summaries</a>]
[]
[<a href="/Sessions/2015/Bills/House/PDF/H1v0.pdf">Filed</a>, <a href="/Sessions/2015/Bills/House/HTML/H1v0.html">HTML</a>, <a href=""></a>, <a></a>]
[<a href="/Sessions/2015/Bills/House/PDF/H1v1.pdf">Edition 1</a>, <a href="/Sessions/2015/Bills/House/HTML/H1v1.html">HTML</a>, <a href=""></a>, <a></a>]
[<a href="/Sessions/2015/Bills/House/PDF/H1v2.pdf">Adopted</a>, <a href="/Sessions/2015/Bills/House/HTML/H1v2.html">HTML</a>, <a href=""></a>, <a></a>]


In [52]:
soup = bs(t.content, 'html.parser')